In [262]:
from icalendar import Calendar, Event
import smtplib
from selenium import webdriver
import os
import time
from datetime import datetime, timezone
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from gcsa.google_calendar import GoogleCalendar
from gcsa.attendee import Attendee
from gcsa.event import Event
from gcsa.serializers.event_serializer import EventSerializer
import pandas as pd
import datetime
import json
import pandas as pd
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from gdoctableapppy import gdoctableapp

# Things to change

In [263]:
#Path to credentials for calendar
CREDENTIALS_PATH_CAL = '/Users/annaglazyrina/Anna_Glazyrina_MSA /Pacing Guide Bot/Calendar_credentials/credentials.json'
#Path to pickle file for calendar (should be the same as credentials)
PICKLE_PATH_CAL = '/Users/annaglazyrina/Anna_Glazyrina_MSA /Pacing Guide Bot/Calendar_credentials/token.pickle'

#Path to credentials for docs
CREDENTIALS_PATH_DOC = '/Users/annaglazyrina/Anna_Glazyrina_MSA /Pacing Guide Bot/Docs_credentials/credentials.json'
#Path to pickle file for docs (should be the same as credentials)
PICKLE_PATH_DOC = '/Users/annaglazyrina/Anna_Glazyrina_MSA /Pacing Guide Bot/Docs_credentials/token.pickle'


# Path to the downloaded calendar file
CAL_PATH = '/Users/annaglazyrina/Downloads/icalexport.ics' #Put path to download folder here
# An URL from moodle (replace with your own)
CAL_URL = "https://moodle-courses2122.wolfware.ncsu.edu/calendar/export_execute.php?userid=317&authtoken=404c7e6e05fb206ee66688be7f5b5a87e1cd1d2d&preset_what=all&preset_time=recentupcoming"

SCOPES = ['https://www.googleapis.com/auth/drive']
#Change document_id accordingly
# DOCUMENT_ID = '1xvwxJyiHhXz94ldmTnaNcz64dgOcgirSxgf70Z7FpvE' # trial doc
DOCUMENT_ID ='15uY1UUXB-mAm1Tja7ESE8MUxU5iCAmcw0FVy4XVGbHI' #public version

#Orange calendar ID
ORANGE_CALENDAR_ID = 'e1q7fa6p1hfq1msodskrg5tvgk@group.calendar.google.com'
#Add Blue Calendar ID here
BLUE_CALENDAR_ID = ''

#Replace ORANGE_CALENDAR_ID with BLUE_CALENDAR_ID
calendar = GoogleCalendar(ORANGE_CALENDAR_ID,credentials_path=CREDENTIALS_PATH_CAL)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=337587925879-2ojgtniijsptsjnkt79g60tre6qgpm14.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=BSigkNdn5WzLjh6jZuSokCFQl4xLAu&access_type=offline


# Google calendar part


In [264]:
# create an empty list of events
events_list = []
# transform every event to json
for event in calendar:
    events_list.append(EventSerializer.to_json(event))
#create an empty list of transformed events
transformed_events=[]
#get the events to the desired format
for event in events_list:
    if event.get('start', 'None')!='None':
        transformed_events.append(
                {
            'Date': event['start'].get('dateTime','          ')[:10],
            'Class': event['summary'],
            'Location': event.get('location','None'),
            'Time': '{}-{}'.format(event['start'].get('dateTime','              ')[11:16], event['end'].get('dateTime','              ')[11:16]),
            'Listen/Read/Watch': '-', #formattable column
            'Participate': '-', #formattable column
            'Submit': '-', #formattable/sourced from moodle
        }
        )
    else:
        #print events that do not have a start date to figure out what to with them
        print(event)
#Create a pandas dataframe of events
events_df = pd.DataFrame(transformed_events)
#remove events that do not have a start date
events_df = events_df[events_df['Date']!='          ']

# Moodle part

In [273]:
# Open chrome browser and download moodle calendar
driver = webdriver.Chrome(ChromeDriverManager().install()) #webdriver.Chrome()
driver.get(CAL_URL)
time.sleep(5)
driver.close()
# Check that the calendar downloaded correctly
if os.path.exists(CAL_PATH):
    #Open the file
    g = open(CAL_PATH,'rb')
    #Read the calendar
    gcal = Calendar.from_ical(g.read())
    #Create an empty list of moodle events
    moodle_events_list = []
    #Format moodle events
    for component in gcal.walk():
        component_dict = {
            "Date":"",
            "Class":"",
#             "Instructor":"",
            "Time":"",
            "Submit":""
        }
        if component.name == "VEVENT":
            component_dict["Class"] = component.get('summary')
            component_dict["Date"] = component.get('dtend').dt.replace(tzinfo=timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d')
            component_dict["Time"] = component.get('dtend').dt.replace(tzinfo=timezone.utc).astimezone(tz=None).strftime('%I:%M%p')
            component_dict["Submit"] = component.get('description')
#             if str(component.get('CATEGORIES')) != 'None':
#                 component_dict["Instructor"] = component.get('CATEGORIES').cats[0]
        moodle_events_list.append(component_dict)
    #Close the file
    g.close()
    #Delete the calendar
    os.remove(CAL_PATH)
# Create a dataframe of all moodle events and apply some transformations
moodle_events_df = pd.DataFrame(moodle_events_list).fillna('-')
moodle_events_df['Submit']=moodle_events_df['Submit'].apply(lambda x: x.strip('\n'))



INFO:WDM:

====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
Current google-chrome version is 92.0.4515
INFO:WDM:Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
INFO:WDM:Get LATEST driver version for 92.0.4515
Driver [/Users/annaglazyrina/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache
INFO:WDM:Driver [/Users/annaglazyrina/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


# Combining the datasets

In [275]:
#Use this if you DON'T have access to moodle
# total_df=events_df.fillna('-')
# Use this if you have access to moodle
total_df = events_df.append(moodle_events_df).fillna('-')

# Saving to file


In [277]:
import json
with open('events_list_orange_with_moodle.json', 'w') as f:
    json.dump(total_df.reset_index(drop=True).to_json(), f)

In [26]:
#Code to read from file above
# import json
# with open('events_list_orange_with_moodle.json', 'r') as f:
#     events_list = json.loads(f.read())
#total_df = pd.read_json(events_list)

,Date,Class,Location,Time,Listen/Read/Watch,Participate,Submit
0,2021-09-21,Leave free before exam,None,09:00-10:45,-,-,-
1,2021-09-22,Leave free before exam,None,09:00-10:45,-,-,-
2,2021-09-23,Leave free before exam,None,10:15-12:00,-,-,-
3,2021-09-24,Leave free before exam,None,10:15-12:00,-,-,-
4,2021-11-30,Program Liaisons meet with Faculty & Staff,TBD,14:45-15:45,-,-,-
...,...,...,...,...,...,...,...
230,2021-09-14,Python - Assignment 2 - Tic Tac Toe is due,None,12:05AM,None,None,Tic Tac Toe
231,2021-09-17,Logistic Regression Quiz 4: Review of Lectures...,None,04:00PM,None,None,
232,2021-09-19,Logistic Regression Quiz 4: Review of Lectures...,None,11:59PM,None,None,
233,2021-09-24,Logistic Regression: Final Assessment opens,None,01:00PM,None,None,


# Logging in to Google Drive (always run)


In [278]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists(PICKLE_PATH_DOC):
    with open(PICKLE_PATH_DOC, 'rb') as token:
        creds = pickle.load(token)
# # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            CREDENTIALS_PATH_DOC, SCOPES)
        creds = flow.run_local_server(host='localhost',port=8080)
    # Save the credentials for the next run
    with open(PICKLE_PATH_DOC, 'wb') as token:
        pickle.dump(creds, token)
        
service = build('docs', 'v1', credentials=creds)

# Build document from start (ERASES ALL COMMENTS and needs to be used on an empty document)

In [287]:
# date_list = list(set(total_df['Date']))
# date_list.sort()

In [290]:
# def build_empty_doc(date_list): 
#     requests = []
#     # Insert text and empty table
#     for guide_date in date_list:
#         text = "This is an automatically created pacing guide for {}".format(guide_date)
#         requests = requests + [
#             #insert text
#             {
#                 'insertText': {
#                     'endOfSegmentLocation': {
#                             'segmentId': ''
#                       },
#                     'text': text
#                 }
#             },
#             #insert empty table
#             {
#                   'insertTable': {
#                       'rows': 1,
#                       'columns': 1,
#                       'endOfSegmentLocation': {
#                             'segmentId': ''
#                       }
#                   }
#             },
#             # insert page break
#              {
#                   'insertPageBreak': {
#                       'endOfSegmentLocation': {
#                             'segmentId': ''
#                       }
#                   },
#             }
#         ]

#     result = service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()
    
# def update_and_clean_all_tables(date_list,total_df):
#     for i in range(0,len(date_list)):
#         guide_date = date_list[i]
#         test_day = total_df[total_df['Date']==guide_date]
#         test_day['range_time']=test_day['Time'].apply(lambda x: True if '-' in x else False)
#         test_day = test_day.sort_values(by=['range_time','Time'])
#         del test_day['range_time']
#         table_data = [list(test_day.columns)] + test_day.values.tolist()
#         resource = {
#                 "oauth2": creds,
#                 "documentId": DOCUMENT_ID,
#                 "tableIndex": i,
#                 "values": table_data
#             }
#         res = gdoctableapp.SetValues(resource)  # You can see the retrieved responses from Docs API.

In [289]:
# build_empty_doc(date_list)
# update_and_clean_all_tables(date_list, total_df)

<ipython-input-288-c222c1b38770>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_day['range_time']=test_day['Time'].apply(lambda x: True if '-' in x else False)


# Update Document ONLY

### TODO: Handle addition of new dates

In [291]:
import re
def format_time_column(sample_time):
    """
    ONLY USE THIS AFTER SORTING
    """
    if '-' in sample_time:
        split_time = re.split('-|:',sample_time)
        for i in range(0,len(split_time)):
            if i in [0,2]:
                if int(split_time[i])>12:
                    new_hour = int(split_time[i])-12
                    if new_hour < 10:
                        split_time[i] = '0{}'.format(new_hour)
                    else:
                        split_time[i] = str(new_hour)
        return '{}:{}-{}:{}'.format(split_time[0],split_time[1],split_time[2],split_time[3])
    else:
        return sample_time
def table_update_prep(current_dict, update_dict):
    moving_dict = {}
    #check that the classes are all the same
    current_classes = set(current_dict.keys())
    update_classes = set(update_dict.keys())
    same_classes = current_classes==update_classes
    update_bool = False
    if same_classes:
        for key in current_dict.keys():
            #Change in time
            if (format_time_column(update_dict[key]['Time'])!=format_time_column(current_dict[key]['Time'])):
                #Replace original time with updated time
                current_dict[key]['Time']=update_dict[key]['Time']
                #Set update to True
                update_bool = True
    if not same_classes:
        #Set update to True
        update_bool = True
        #check if classes were only added or if some were removed
        #removed classes
        in_current_not_in_update = list(current_classes - update_classes)
        #added classes
        in_update_not_in_current = list(update_classes - current_classes)
        if in_current_not_in_update:
            for key in in_current_not_in_update:
                moving_dict[key]=current_dict[key]
                current_dict.pop(key)
            #search for the class in other days and move the comments if it was rescheduled
        if in_update_not_in_current:
            #add rows to table
            for key in in_update_not_in_current:
                current_dict[key]=update_dict[key]
    return update_bool, current_dict, moving_dict

In [293]:
# Load updated calendar data
# import json
# with open('events_list_orange_with_moodle.json', 'r') as f:
#     events_list = json.loads(f.read())
# update_df=pd.read_json(events_list)
# del update_df['index']

# Use this instead because it's all in the same notebook
update_df = total_df

#Dictionary for moved comments
comments_to_move = {}
#Dictionary for table indices and dates - need this to insert events in between dates later
index_date_dict = {}

#Looking at all of the tables
resource = {
    "oauth2": creds,
    "documentId": DOCUMENT_ID,
    # "showAPIResponse": True
}
res = gdoctableapp.GetTables(resource) # You can see the retrieved values like this.

#Compare each table to see if it needs updating
for table_index  in range(0,len(res['tables'])):
    try:
        #extract the table that is currently in the document
        current_table = res['tables'][table_index]['values']
        #transform the current table into a pandas dataframe
        day_table_current = pd.DataFrame(current_table[1:],columns=current_table[0]).drop_duplicates()
        #extract the date corresponding to that dataframe
        current_date = max(day_table_current['Date'])
        #update the dictionary for table indices and dates
        index_date_dict.update({table_index:current_date})
        #compare the date to today's date - today's events are not updated (FIGURE OUT A FIX TO THAT)
        if current_date <= datetime.datetime.today().strftime('%Y-%m-%d'):
            print(current_date)
            print('This date has passed - no updates')
        else:
            #extract the table that was passed from google calendar + Moodle for this date
            day_table_new = update_df[update_df['Date']==current_date].drop_duplicates()
            #Transform both tables to dictionaries
            current_dict = day_table_current.set_index('Class').to_dict('index')
            update_dict = day_table_new.set_index('Class').to_dict('index')
            #Figure out if the table needs updating, create a new table, 
            #store the events that were moved form this day
            update_bool, current_dict, moving_dict = table_update_prep(current_dict, update_dict)
            #If the table need updating
            if update_bool == True: # add "or update_bool == False" to update all
                print(current_date)
                print(current_dict)
                #Transform the dictionary to a sorted dataframe
                cols=['Date','Class','Location', 'Time', 'Listen/Read/Watch', 'Participate','Submit']
                new_df = pd.DataFrame.from_dict(current_dict,'index').reset_index().rename(columns={'index':'Class'})[cols]
                #create a temporary variable that checks whether the time variable is a range or not
                new_df['range_time']=new_df['Time'].apply(lambda x: True if '-' in x else False)
                #Put the non-range variables in the beginning of the document
                new_df = new_df.sort_values(by=['range_time','Time'])
                del new_df['range_time']
                #Format time to a 12-hour standard
                new_df['Time'] = new_df['Time']#.apply(lambda x: format_time_column(x))
                #Convert the data to a postable format
                table_data = [list(new_df.columns)] + new_df.values.tolist()
                #If there was an event moved from this day, delete a correxponding number of row
                if moving_dict:
                    resource = {
                        "oauth2": creds,
                        "documentId": DOCUMENT_ID,
                        "tableIndex": table_index,
                        "deleteRows": list(range(0,len(moving_dict))),  # Start index is 0.
                        "deleteColumns": [0],
                    }
                    res_2 = gdoctableapp.DeleteRowsAndColumns(resource)
                    #Move the info to a separate dictionary to consider later
                    comments_to_move.update(moving_dict)
                #Update the table values
                resource = {
                    "oauth2": creds,
                    "documentId": DOCUMENT_ID,
                    "tableIndex": table_index,
                    "values": table_data
                }
                res_2 = gdoctableapp.SetValues(resource)  # You can see the retrieved responses from Docs API.
    except ValueError:
        print('ValueError')
        
#Check if there were any new dates added
update_dates = set(update_df['Date'])
current_dates = set(index_date_dict.values())
new_dates = list(update_dates-current_dates)
new_dates = [x for x in new_dates if x !='']
new_dates.sort()
n_added = 0
for i in range(0,len(new_dates)):
    sample_added_date = new_dates[i]
    print(sample_added_date)
    #do not add dates before start
    if len([x for x in index_date_dict.values() if x<sample_added_date]) != 0:
        #see how many tables come before your new table (+i to account for tables added)
        tables_before_new = len([x for x in index_date_dict.values() if x<sample_added_date]) + n_added
        #Looking at all of the tables
        resource = {
            "oauth2": creds,
            "documentId": DOCUMENT_ID,
            # "showAPIResponse": True
        }
        res = gdoctableapp.GetTables(resource)
        end_index = [x['tablePosition']['endIndex'] for x in res['tables'] if x['index']==tables_before_new-1][0]
        text = "This is an automatically created pacing guide for {}".format(sample_added_date)
        len_text = len(text)
        requests = [
        #         insert page break
                 {
                      'insertPageBreak': {
                          'location': {
                                'index': end_index
                          }
                      },
                },
        #       insert text
                {
                    'insertText': {
                        'location': {
                                'index': end_index+1
                          },
                        'text': text
                    }
                },
        #       insert empty table
                {
                      'insertTable': {
                          'rows': 1,
                          'columns': 1,
                          'location': {
                                'index': end_index+len_text+1
                          }
                      }
                },
        ]
        # Create a new page
        result = service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()
        # Add values to the table
        test_day = update_df[update_df['Date']==sample_added_date]
        test_day['range_time']=test_day['Time'].apply(lambda x: True if '-' in x else False)
        test_day = test_day.sort_values(by=['range_time','Time'])
        del test_day['range_time']
        test_day['Time'] = test_day['Time']#.apply(lambda x: format_time_column(x))
        table_data = [list(test_day.columns)] + test_day.values.tolist()
        resource = {
                "oauth2": creds,
                "documentId": DOCUMENT_ID,
                "tableIndex": tables_before_new,
                "values": table_data
            }
        res_2 = gdoctableapp.SetValues(resource)  # You can see the retrieved responses from Docs API.
        n_added+=1
#Update the index to date dictionary
for table_index  in range(0,len(res['tables'])):
    #extract the table that is currently in the document
    current_table = res['tables'][table_index]['values']
    #transform the current table into a pandas dataframe
    day_table_current = pd.DataFrame(current_table[1:],columns=current_table[0]).drop_duplicates()
    #extract the date corresponding to that dataframe
    current_date = max(day_table_current['Date'])
    #update the dictionary for table indices and dates
    index_date_dict.update({table_index:current_date})
#Working through comments for the events that were moved
#Looking at all of the tables
resource = {
    "oauth2": creds,
    "documentId": DOCUMENT_ID,
    # "showAPIResponse": True
}
res = gdoctableapp.GetTables(resource) # You can see the retrieved values like this.
#see if the event shows up in the rest of the update dataframe
for class_name in comments_to_move.keys():
    if class_name in list(update_df['Class']):
        date_to_move_to = max(update_df[update_df['Class']==class_name]['Date'])
        print(date_to_move_to)
        index_to_move_to = [index for index, date in index_date_dict.items() if date == date_to_move_to][0]
        current_table = res['tables'][index_to_move_to]['values']
        day_table_current = pd.DataFrame(current_table[1:],columns=current_table[0]).drop_duplicates()
        current_dict = day_table_current.set_index('Class').to_dict('index')
        keys_to_keep = ['Listen/Read/Watch','Participate','Submit']
        comments_to_keep = {key: comments_to_move[class_name][key] for key in keys_to_keep}
        current_dict[class_name].update(comments_to_keep)
        #Transform the dictionary to a sorted dataframe
        cols=['Date','Class','Location', 'Time', 'Listen/Read/Watch', 'Participate','Submit']
        new_df = pd.DataFrame.from_dict(current_dict,'index').reset_index().rename(columns={'index':'Class'})[cols]
        #create a temporary variable that checks whether the time variable is a range or not
        new_df['range_time']=new_df['Time'].apply(lambda x: True if '-' in x else False)
        #Put the non-range variables in the beginning of the document
        new_df = new_df.sort_values(by=['range_time','Time'])
        del new_df['range_time']
        #Format time to a 12-hour standard
        new_df['Time'] = new_df['Time']#.apply(lambda x: format_time_column(x))
        #Convert the data to a postable format
        table_data = [list(new_df.columns)] + new_df.values.tolist()
        resource = {
                    "oauth2": creds,
                    "documentId": DOCUMENT_ID,
                    "tableIndex": index_to_move_to,
                    "values": table_data
                }
        res_2 = gdoctableapp.SetValues(resource)



This date has passed - no updates
2021-08-20
This date has passed - no updates
2021-08-24
This date has passed - no updates
ValueError
ValueError
ValueError
ValueError
